![](https://img.shields.io/badge/PO.DAAC-Contribution-%20?color=grey&labelColor=blue)

> Matériel provenant du PO.DAAC Cookbook, cliquez sur [ce lien](https://github.com/podaac/tutorials/blob/master/notebooks/datasets/SWOTHR_localmachine.ipynb) pour accéder à la version GitHub du carnet.

# Exploration des données hydrologiques SWOT à partir d'une machine locale

## Accès et visualisation des ensembles de données SWOT

### Prérequis:
Environnement de calcul local, par exemple un ordinateur portable.

### Objectifs d'apprentissage:
- Accéder aux données SWOT HR archivés dans le nuage NASA Earthdata en les téléchargeant sur une machine locale.
- Visualiser les données pour une vérification rapide.

#### SWOT Level 2 KaRIn High Rate Version 2.0 Datasets:

1. **River Vector Shapefile** - SWOT_L2_HR_RIVERSP_2.0

2. **Lake Vector Shapefile** - SWOT_L2_HR_LAKESP_2.0

3. **Water Mask Pixel Cloud NetCDF** - SWOT_L2_HR_PIXC_2.0

4. **Water Mask Pixel Cloud Vector Attribute NetCDF** - SWOT_L2_HR_PIXCVec_2.0

5. **Raster NetCDF** - SWOT_L2_HR_Raster_2.0

6. **Single Look Complex Data product** - SWOT_L1B_HR_SLC_2.0

_Ce carnet a été modifié et traduit par l'équipe de l'Université de Sherbrooke et de l'Université Laval. Auteurs originaux :  Cassie Nickles, NASA PO.DAAC (Feb 2024) || Autres contributeurs : Zoe Walschots (PO.DAAC Summer Intern 2023), Catalina Taglialatela (NASA PO.DAAC), Luis Lopez (NASA NSIDC DAAC)*._

_Dernière mise à jour : 10 septembre 2024_

  

### Librairies nécessaires

In [ ]:
!pip install contextily
!pip install earthaccess
!pip install --upgrade holoviews hvplot
!pip install holoviews hvplot bokeh xarray
!pip install rioxarray
!pip install rasterio
!pip install shapely
!pip install geoviews
!pip install pyproj

#!pip install hvplot


import glob
import h5netcdf
import xarray as xr
import pandas as pd
import geopandas as gpd
import contextily as cx
import numpy as np
import matplotlib.pyplot as plt
import hvplot.xarray
import holoviews as hv
import zipfile
import earthaccess
import os
import rioxarray
from shapely.geometry import mapping
import csv
import shapefile
import geoviews as gvts
from pyproj import Proj
import logging


### Connexion à Earthdata

Un compte Earthdata est nécessaire pour accèder aux données du système Earthdata de la NASA. Si vous n'en avez pas encore, rendez-vous sur https://urs.earthdata.nasa.gov pour créer votre compte. La création du compte est gratuite et ne prend que quelques minutes. Nous utilisons `earthaccess` pour vous identifier.


In [ ]:
auth = earthaccess.login()

### Accès à un fichier


#### **1. River Vector Shapefiles**

Le lien d'accès https peut être trouvé en utilisant earthaccess data search. Cette collection est composée de fichiers Reach et Node.

Earthdata Search [(voir le tutoriel)](https://nasa-openscapes.github.io/2021-Cloud-Workshop-AGU/tutorials/01_Earthdata_Search.html) peut également être utilisé pour effectuer une recherche manuelle à partir d'une interface graphique.

Pour des conseils supplémentaires concernant la recherche spatiale des données SWOT HR L2, voir également [PO.DAAC Cookbook - SWOT Chapter tips section](https://podaac.github.io/tutorials/quarto_text/SWOT.html#tips-for-swot-hr-spatial-search).



#### Recherche des données d'intérêt

In [ ]:
#Recupération des granules ayant les caractéristiques voulues en passant par la fonction `earthdata.search_data`
river_results = earthaccess.search_data(short_name = 'SWOT_L2_HR_RIVERSP_2.0',
                                        #temporal = ('2024-02-01 00:00:00', '2024-02-29 23:59:59'), # peut également être filtré en fonction de la plage temporelle
                                        granule_name = '*Node*_576_NA*') # ici nous filtrons par fichiers Nodes (pas Reach), par passe et par continent
                                                                         # spécifier 'Node' ou 'Reach' selon le fichier voulu

In [ ]:
#Imprimer les caractéristiques des granules associés à la passe sélectionnée.
print(river_results)

#### Téléchargez, décompressez et lisez les données

Télédéchargeons le fichier de données sélectionné ! `earthaccess.download` a une liste comme format d'entrée, nous devons donc mettre des crochets autour du fichier que nous voulons.



In [ ]:
earthaccess.download([river_results[3]], "./data_downloads")

Le format natif de ces données est un fichier .zip et nous voulons le fichier .shp dans ce fichier .zip. Nous devons donc extraire les données pour les ouvrir. Nous allons récupérer le nom du fichier que nous venons de télécharger, puis extraire toutes les données dans le dossier `data_downloads`.

In [ ]:
filename = earthaccess.results.DataGranule.data_links(river_results[3], access='external')
filename = filename[0].split("/")[-1]
filename

In [ ]:
with zipfile.ZipFile(f'data_downloads/{filename}', 'r') as zip_ref:
    zip_ref.extractall('data_downloads')

Ouverture du fichier shapefile avec `geopandas`

In [ ]:
filename_shp = filename.replace('.zip','.shp')

In [ ]:
SWOT_HR_shp1 = gpd.read_file(f'data_downloads/{filename_shp}')

#afficher la table d'attributs
SWOT_HR_shp1

#### Affichage des données des rivières SWOT

In [ ]:
# Affichage simple
fig, ax = plt.subplots(figsize=(10,10))
SWOT_HR_shp1.plot(ax=ax, color='black')
ax.set_ylim(47.08,47.56) #sélectionnez votre zone d'intérêt pour un meilleur apeçu
ax.set_xlim(-72.91,-72.60) #sélectionnez votre zone d'intérêt pour un meilleur apeçu
cx.add_basemap(ax, crs=SWOT_HR_shp1.crs, source=cx.providers.Esri.WorldStreetMap)

In [ ]:
# Il est également possible d'afficher les données avec la fonction `explore` de geopandas
#SWOT_HR_shp1.explore()

#### **2. Lake Vector Shapefiles**

Les fichiers vectoriels des lacs sont accessibles de la même manière que les fichiers vectoriels des rivières ci-dessus.

Pour des conseils supplémentaires sur la recherche spatiale des données SWOT HR L2, voir également [PO.DAAC Cookbook - SWOT Chapter tips section](https://podaac.github.io/tutorials/quarto_text/SWOT.html#tips-for-swot-hr-spatial-search).

#### Recherche des données d'intérêt

In [ ]:
lake_results = earthaccess.search_data(short_name = 'SWOT_L2_HR_LAKESP_2.0',
                                        #temporal = ('2024-02-01 00:00:00', '2024-02-29 23:59:59'), # peut également être filtré en fonction de la plage temporelle
                                        granule_name = '*Prior*_576_NA*') # ici nous filtrons les fichiers avec 'Prior' (Cette collection a trois options : Obs, Unassigned, et Prior), par passe et par continent

In [ ]:
#Imprimer les caractéristiques des granules associés à la passe sélectionnée.
print(lake_results)

Téléchargez le fichier de données sélectionné ! earthaccess.download a une liste comme format d'entrée, nous devons donc mettre des crochets autour du fichier que nous voulons.

In [ ]:
earthaccess.download([lake_results[0]], "./data_downloads")

Le format natif de ces données est un fichier .zip et nous voulons le fichier .shp dans ce fichier .zip. Nous devons donc extraire les données pour les ouvrir. Nous allons récupérer le nom du fichier que nous venons de télécharger, puis extraire toutes les données dans le dossier `data_downloads`.

In [ ]:
filename2 = earthaccess.results.DataGranule.data_links(lake_results[0], access='external')
filename2 = filename2[0].split("/")[-1]
filename2

In [ ]:
with zipfile.ZipFile(f'data_downloads/{filename2}', 'r') as zip_ref:
    zip_ref.extractall('data_downloads')

Ouverture du shapefile avec `geopandas`

In [ ]:
filename_shp2 = filename2.replace('.zip','.shp')
filename_shp2

In [ ]:
SWOT_HR_shp2 = gpd.read_file(f'data_downloads/{filename_shp2}')

#Afficher la table d'attributs
SWOT_HR_shp2

#### Affichage des données des lacs SWOT

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))
SWOT_HR_shp2.plot(ax=ax, color='black')
ax.set_ylim(46.68,46.85) #sélectionnez votre zone d'intérêt pour un meilleur apeçu  (Exemple : lac Taureau)
ax.set_xlim(-74.06,-73.52) #sélectionnez votre zone d'intérêt pour un meilleur apeçu
cx.add_basemap(ax, crs=SWOT_HR_shp2.crs, source=cx.providers.Esri.WorldStreetMap)


L'accès aux fichiers ci-dessous est différent de celui des fichiers .shp présentés jusqu'à maintenant. Puisque les collections SWOT HR suivantes sont stockées sous forme de fichiers **netCDF** dans le nuage, nous n'avons pas besoin d'extraire les shapefiles d'un fichier zip. Pour le reste des produits, nous les ouvrirons via `xarray` et non `geopandas`.

#### **3. Water Mask Pixel Cloud NetCDF**

#### Recherche des données de la collection selon les caractéristiques voulues

In [ ]:
pixc_results = earthaccess.search_data(short_name = 'SWOT_L2_HR_PIXC_2.0',
                                        #temporal = ('2024-02-01 00:00:00', '2024-02-29 23:59:59'), # filtre en fonction de la plage temporelle
                                        granule_name = '*_576_073L*') # numéro de pass, de tuile et côté de fauchée (R ou L)
                                        #bounding_box = (-72.73,46.58,-72.60,46.62)) # filtre par boîte englobante, pour trouver votre boîte englobante : http://bboxfinder.com/

In [ ]:
#Imprimer les caractéristiques des granules associés à la passe, à la tuile et à la fauchée.
print(pixc_results)

Téléchargez le fichier de données sélectionné ! earthaccess.download a une liste comme format d'entrée, nous devons donc mettre des crochets autour du fichier que nous voulons.

In [ ]:
earthaccess.download([pixc_results[1]], "./data_downloads")

#### Ourvrir les données avec xarray





Ces fichiers netCDF sont formatés en trois groupes intitulés "pixel cloud", "tvp", ou "noise" (plus de détails [ici](https://podaac-tools.jpl.nasa.gov/drive/files/misc/web/misc/swot_mission_docs/pdd/D-56411_SWOT_Product_Description_L2_HR_PIXC_20200810.pdf)). Afin d'accéder aux coordonnées et aux variables du fichier, un groupe doit être spécifié lors de l'appel à xarray open_dataset.

In [ ]:
ds_PIXC = xr.open_mfdataset("data_downloads/SWOT_L2_HR_PIXC_*.nc", group = 'pixel_cloud', engine='h5netcdf') #Si plusieurs fichiers PIXC sont téléchargés dans le fichier data_download, spécifiez lequel des fichiers doit être affiché.
ds_PIXC


#### Affichage des données

In [ ]:
# L'affichage peut prendre quelques minutes (environ 5 minutes)
fig, ax = plt.subplots(figsize=(10, 10))
cax=plt.scatter(x=ds_PIXC.longitude, y=ds_PIXC.latitude, c=ds_PIXC.height, s=1) #ajustement de la taille des pixels avec s= taille désirée
cbar = fig.colorbar(cax, ax=ax, shrink=0.5)
cbar.set_label('Height (m)')

# ajustement de la barre de couleur
cbar.ax.set_aspect('auto')

cx.add_basemap(ax, crs='EPSG:4326', source=cx.providers.Esri.WorldStreetMap)




#### Coupez et convertissez le PIXC en un fichier .shp pour permettre son ouverture dans QGIS.

In [ ]:
from shapely.geometry import Point, Polygon


logging.getLogger('fiona').setLevel(logging.ERROR)

lat_min = 46.58
lat_max = 46.62
lon_min = -72.73
lon_max = -72.60

lat = np.asarray(ds_PIXC.latitude[:])
lon = np.asarray(ds_PIXC.longitude[:])
classif  = np.asarray(ds_PIXC.classification[:])

mask = (lat > lat_min) & (lat < lat_max) & (lon > lon_min) & (lon < lon_max) & (classif>2) & (classif<5)
#mask = (lat > lat_min) & (lat < lat_max) & (lon > lon_min) & (lon < lon_max)

data = {}
exclude_vars = ['interferogram', 'illumination_time','illumination_time_tai','pixc_line_qual', 'pixc_line_to_tvp','data_window_first_valid','data_window_last_valid','data_window_first_cross_track','data_window_last_cross_track']
for var_name in ds_PIXC.variables:
    if var_name not in exclude_vars:
        var_data = np.asarray(ds_PIXC.variables[var_name][:])
        var_data = var_data[mask]
        if len(var_data.shape) > 1:
            var_data = var_data.reshape(-1)
        data[var_name] = var_data

df = pd.DataFrame(data)

points = [Point(x, y) for x, y in zip(df.longitude, df.latitude)]

gdf_out = gpd.GeoDataFrame(df, geometry=points, crs="EPSG:4326")
out_shp= './data_downloads/SWOT_L2_HR_PIXC_003_576_073L_20230921T121803_20230921T121814_PGC0_01.shp'
gdf_out.to_file(out_shp)


In [ ]:
# Affichez vos données PIXC coupées. Vous pouvez aussi les télécharger pour les visualiser dans QGIS.
fig, ax = plt.subplots(figsize=(10, 10))
cax= plt.scatter(x=gdf_out.longitude, y=gdf_out.latitude, c=gdf_out.height, s=1) #ajustement de la taille des pixels avec s= taille désirée
cbar = fig.colorbar(cax, ax=ax, shrink=0.5)
cbar.set_label('Height (m)')

# ajustement de la barre de couleur
cbar.ax.set_aspect('auto')

cx.add_basemap(ax, crs='EPSG:4326', source=cx.providers.Esri.WorldStreetMap)



#### **4. Water Mask Pixel Cloud Vector Attribute NetCDF**

#### Recherche des données d'intérêt

In [ ]:
pixcvec_results = earthaccess.search_data(short_name = 'SWOT_L2_HR_PIXCVEC_2.0',
                                        #temporal = ('2024-02-01 00:00:00', '2024-02-29 23:59:59'), # peut également être filtré en fonction de la plage horaire
                                        granule_name = '*_576_073L*') # numéro de pass, de tuile et côté de fauchée (R ou L)
                                        #bounding_box = (-72.73,46.58,-72.60,46.62)) # filtre par boîte englobante, pour trouver votre boîte englobante : http://bboxfinder.com/


In [ ]:
#Imprimer les caractéristiques des granules
print(pixcvec_results)

Téléchargez le fichier de données sélectionné ! earthaccess.download a une liste comme format d'entrée, nous devons donc mettre des crochets autour du fichier que nous voulons.

In [ ]:
earthaccess.download([pixcvec_results[0]], "./data_downloads")

#### Ouverture des données avec xarray

Nous allons chercher automatiquement le nom du fichier que nous venons de télécharger, puis nous allons l'afficher avec `xarray`.

In [ ]:
ds_PIXCVEC = xr.open_mfdataset("data_downloads/SWOT_L2_HR_PIXCVec_*.nc", decode_cf=False,  engine='h5netcdf')
ds_PIXCVEC

#### Affichage rapide

In [ ]:
pixcvec_htvals = ds_PIXCVEC.height_vectorproc.compute()
pixcvec_latvals = ds_PIXCVEC.latitude_vectorproc.compute()
pixcvec_lonvals = ds_PIXCVEC.longitude_vectorproc.compute()

#Avant d'afficher le graphique, les valeurs de remplissage sont fixées à NaN (Not a Number) afin d'améliorer l'affichage du graphique dans l'espace.
pixcvec_htvals[pixcvec_htvals > 15000] = np.nan
pixcvec_latvals[pixcvec_latvals < 1] = np.nan
pixcvec_lonvals[pixcvec_lonvals > -1] = np.nan


In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
plt.scatter(x=pixcvec_lonvals, y=pixcvec_latvals, c=pixcvec_htvals, s=1) #ajustement de la taille des pixels avec s= taille désirée
plt.colorbar().set_label('Height (m)')

cx.add_basemap(ax, crs='EPSG:4326', source=cx.providers.Esri.WorldStreetMap)
plt.show()

#### **5. Raster NetCDF**

#### Recherche des données d'intérêt

In [ ]:
raster_results = earthaccess.search_data(short_name = 'SWOT_L2_HR_Raster_2.0',
                                        #temporal = ('2024-02-01 00:00:00', '2024-02-29 23:59:59'), # peut également être filtré en fonction de la plage temporelle
                                        #bounding_box = (-72.73,46.58,-72.60,46.62), # filtre par boîte englobante, pour trouver votre boîte englobante : http://bboxfinder.com/
                                        granule_name = '*100m*_576_037F*') # ici nous filtrons les données avec la mention '100m' (Cette collection à deux options de résolution: 100m & 250m)



In [ ]:
#Imprimer les caractéristiques des granules associées à la passe, à la scène et à la résolution sélectionnées.
print(raster_results)

Téléchargons un fichier de données

In [ ]:
earthaccess.download([raster_results[1]], "./data_downloads")

#### Ouverture des données avec xarray

Nous allons chercher automatiquement le nom du fichier que nous venons de télécharger, puis nous allons l'afficher avec `xarray`.

In [ ]:
ds_raster = xr.open_mfdataset(f'data_downloads/SWOT_L2_HR_Raster*', engine='h5netcdf')
ds_raster

#### Affichage intéractif des données avec `hvplot`

In [ ]:
hv.extension('bokeh', 'matplotlib')
plot = ds_raster['wse'].hvplot.image(y='y', x='x')
hv.output(plot)



#### Masquer une variable selon son indicateur de qualité
Exemple pour une donnée L2_HR_Raster, indicateur "wse_qual":\
0 = bonne\
1 = suspecte -  peut comporter des erreurs importantes\
2 = dégradée - susceptible de contenir des erreurs importantes\
3 = mauvaise - peut ne pas faire de sens et doit être ignorée

In [ ]:
variable_to_mask = ds_raster['wse']
mask_variable = ds_raster['wse_qual']


In [ ]:
# Définir la condition pour masquer les données selon l'indicateur de qualité
mask_condition = mask_variable < 3
masked_variable = variable_to_mask.where(mask_condition)
masked_variable


In [ ]:
# Mise à jour de la variable masquée dans l'ensemble de données
hv.extension('bokeh', 'matplotlib')
plot2 = masked_variable.hvplot.image(y='y', x='x')
hv.output(plot2)


#### Découpez vos données Raster NetCDF masquées

In [ ]:
# Créez un fichier pour vos données découpées
os.makedirs('./content/clip_data', exist_ok=True)

In [ ]:
#Définir la région d'intérêt
from shapely.geometry import box

ROI = box(-72.73,46.58,-72.60,46.62)
bbox_gdf = gpd.GeoDataFrame({'geometry': [ROI]}, crs='EPSG:4326')


In [ ]:
# Définir les dimensions spatiales du jeu de données et le système de référence des coordonnées (crs).
masked_variable.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=True)
masked_variable.rio.write_crs("epsg:32618", inplace=True)


In [ ]:
# Découper le raster
# Si nécessaire, reprojetez d'abord la région d'intérêt dans le même EPSG que celui du netCDF
bbox_gdf = bbox_gdf.to_crs("epsg:32618")
clipped = masked_variable.rio.clip(bbox_gdf.geometry.apply(mapping), drop=True)

#Vérification de la zone d'intérêt recadrée
hv.extension('bokeh', 'matplotlib')
plot2 = clipped.hvplot.image(y='y', x='x')
hv.output(plot2)


In [ ]:
# Supprimez l'attribut 'grid_mapping'
if 'grid_mapping' in clipped.attrs:
    del clipped.attrs['grid_mapping']

# Sauvegardez le raster découpé en un fichier NetCDF
clipped_path = './content/clip_data/clipped_raster.nc'  # Définir le chemin et le nom du fichier à sauvegarder
clipped.to_netcdf(clipped_path)

#### **6. Transformation des systèmes de références**





Le système de référence de SWOT n'est pas le même que celui du Canada. Il est donc nécessaire de convertir les données.
**Attention : Chaque agence géodésique provinciale canadienne peut adopter un système de référence vertical et une époque différents !**


Découper les données avec le masque du Québec (nécessaire car chaque province peut adopter une époque et un système de référence vertical différents).

Attention : changez le nom du fichier RiverSp pour qu'il corresponde à celui que vous voulez.


In [ ]:
import os
os.makedirs('ref_change', exist_ok=True)

# Téléchargez le fichier GeoJSON avec les provinces
url = 'https://data.opendatasoft.com/api/explore/v2.1/catalog/datasets/georef-canada-province@public/exports/geojson?lang=fr&timezone=America%2FNew_York'

gdf_prov = gpd.read_file(url)

# Filtrez le GeoDataFrame pour sélectionner seulement celui de la province de Québec
quebec_gdf = gdf_prov[gdf_prov['prov_name_fr'] == 'Québec']

print(quebec_gdf.geometry)


In [ ]:
#  Téléchargez le shapefile à découper et vérifiez que le masque et le fichier RiverSp on le même système de coordonnées
from shapely.ops import unary_union

logging.getLogger('fiona').setLevel(logging.ERROR)

shapefile_path = '/content/data_downloads/SWOT_L2_HR_RiverSP_Node_005_576_NA_20231102T054530_20231102T054541_PGC0_01.shp'

points_gdf = gpd.read_file(shapefile_path)

quebec_gdf = quebec_gdf.to_crs(points_gdf.crs)

points_gdf_masked = points_gdf.clip(mask= quebec_gdf.geometry)

# Sauvergardez les noeuds selectionnés en format shapefile
points_gdf_masked.to_file('/content/ref_change/qc_nodes.shp')



Affichage des données découpées

In [ ]:
# Affichage rapide
fig, ax = plt.subplots(figsize=(7,5))
points_gdf_masked.plot(ax=ax, color='black')
quebec_gdf.boundary.plot(ax=ax, edgecolor='red')
cx.add_basemap(ax, crs=points_gdf_masked.crs, source=cx.providers.Esri.WorldStreetMap)
plt.show()

#Sauvegardez votre image
plt.savefig('/content/clippeddata.png')

Ensuite, les données doivent être extraites et sauvegardées dans le bon format pour être utilisées avec l'outil TRX de RNCan.

In [ ]:
# Entrez le chemin de vos données
data_all = []

with shapefile.Reader('/content/ref_change/qc_nodes.shp') as shp:

    fields = [field[0] for field in shp.fields if field[0] != 'DeletionFlag']

    for record in shp.records():
          attributs = dict(zip(fields, record))
          #Selectionnez le node_id, la latitude, la longitude et les wse qui ne sont pas -999999999999
          if all(value != -999999999999 for value in attributs.values()):
            data_all.append([attributs['node_id'],attributs['lat'],attributs['lon'],attributs['wse']])

header = ['Station','latitude','longitude','height']

# Création et écriture d'une fichier CSV
with open('/content/ref_change/wse_qc.csv', 'w', newline='', encoding='utf-8') as fichier:
    writer = csv.writer(fichier)
    writer.writerow(header)

    # Écriture des données
    for data in data_all:
        writer.writerow(data)

#Sauvergarde dans les fichiers
from google.colab import files
files.download("/content/ref_change/wse_qc.csv")

Enfin, pour utiliser TRX en ligne, rendez-vous sur le site web : https://webapp.csrs-scrs.nrcan-rncan.gc.ca/geod/tools-outils/trx.php?locale=fr


Remplissez cette information dans la fenêtre **Traitement par lots** :

**Origine** :  Cadre de référence : ITRF2014 epoque 01/01/2010

**Destination** : Cadre de référence : NAD83(CSRS) Coordinates : Geographic

 **Transformation d'époque** : 01/01/1997 (époque adoptée pour le Québec, adaptez l'époque en conséquence)

Sélectionnez le fichier téléchargé précédemment.

#### **7. Téléchargez toutes vos données SWOT sur votre machine locale d'un coup**






Vous pouvez télécharger les données une par une en cliquant sur les trois petits points à droite du nom du fichier et ensuite sur « Télécharger ». Pour télécharger toutes les données d'un coup, exécutez les cellules suivantes. Les temps d'exécution et de téléchargement peuvent être longs.

In [ ]:
# Créez un fichier zip avec toutes les données et choisissez le fichier à compresser
!zip -r /content/data.zip /content/data_downloads

In [ ]:
# Téléchargez le fichier compressé
from google.colab import files
files.download("/content/data.zip")


#### **8. Travailler avec HYDROCON**


Extraire des séries temporelles avec Hydrocon pour les Reach et Nodes d'intérêt

In [ ]:
from ast import And
import folium
import requests
from io import StringIO
import pandas as pd
import matplotlib.pyplot as plt

# Vous pouvez choisir un identifiant de Node ou de Reach à partir du produit RiverSP obtenu dans la section 1.
#Exemple pour la rivière Saint-Maurice

#feature='Node'
#feature_id="71250300150401"
feature="Reach"
feature_id="72520100041"
#start_time="2023-08-01T00:00:00Z"
#end_time="2024-08-01T00:00:00Z"
#fields=reach_id,time_str,wse,width

parameters = "https://soto.podaac.earthdatacloud.nasa.gov/hydrocron/v1/timeseries?feature="+feature+"&feature_id="+ feature_id +"&start_time=2024-01-01T00:00:00Z&end_time=2024-06-06T00:00:00Z&output=geojson&fields=reach_id,time_str,wse,width,cycle_id"


hydrocron_response = requests.get(
    parameters
).json()

hydrocron_response

# Extraction du geojson pour l'afficher sur la carte

geojson_data = hydrocron_response['results']['geojson']

geojson_data

# Configurez la carte à l'aide de Folium (https://python-visualization.github.io/folium/latest/)

map = folium.Map (zoom_start=13, tiles="cartodbpositron", width=700, height=700)

# Ajoutez le geojson provenant d'Hydrocron à la carte
folium.GeoJson(geojson_data, name='SWOT River Reach').add_to(map)
folium.LayerControl().add_to(map)

# Centrez sur la rivière
map.fit_bounds(map.get_bounds(), padding=(5, 5))

map


Afficher une série temporelle de niveau d'eau pour un noeud d'intérêt

In [ ]:
#Exemple pour la rivière Saint-Maurice

feature='Node'
feature_id="72520100230111" # Rapides-Manigance
start_time="2023-08-01T00:00:00Z"
end_time="2024-08-01T00:00:00Z"
#fields=reach_id,time_str,wse,width

#parameters = "https://soto.podaac.earthdatacloud.nasa.gov/hydrocron/v1/timeseries?feature="+feature+"&feature_id="+ feature_id +"&start_time=2024-01-01T00:00:00Z&end_time=2024-06-14T00:00:00Z&output=csv&fields=reach_id,node_id,time_str,node_q,wse,width,cycle_id"

parameters = "https://soto.podaac.earthdatacloud.nasa.gov/hydrocron/v1/timeseries?feature="+feature+"&feature_id="+ feature_id +"&start_time="+start_time+"&end_time="+end_time+"&output=csv&fields=reach_id,node_id,time_str,node_q,wse,width,cycle_id"



hydrocron_response = requests.get(
    parameters
).json()

hydrocron_response
csv_str = hydrocron_response['results']['csv']
df = pd.read_csv(StringIO(csv_str))
ind = df.node_q<3

df = df[df['time_str'] != 'no_data']
df.time_str = pd.to_datetime(df.time_str, format='%Y-%m-%dT%H:%M:%SZ')
fig = plt.figure(figsize=(15,5))
plt.plot(df.time_str[ind], df.wse[ind], marker='o')

plt.ylabel('Water surface elevation (m)')
plt.xlabel('SWOT observation date')
plt.title('Water Surface Elevation from Hydrocron for Node: ' + str(df.node_id[0]))

#Sauvegardez votre image
plt.savefig('/content/WSE_Hydrocon.png')


#### **9. Extraire un profil entre deux noeuds**

In [ ]:
# Créez un fichier pour vos données
os.makedirs('./content/profil_node', exist_ok=True)

Authentification et téléchargement des données

In [ ]:
# Fonction pour rechercher et télécharger les données
def download_data(pass_numbers, continent_code, path, temporal_range):
    links_list = []
    for pass_num in pass_numbers:
        river_results = earthaccess.search_data(
            short_name='SWOT_L2_HR_RIVERSP_2.0',
            temporal=temporal_range,
            granule_name=f"*Node*_{pass_num}_{continent_code}*"
        )
        links_list.extend([earthaccess.results.DataGranule.data_links(result, access='external')[0]
                           for result in river_results])

    # Téléchargement des fichiers
    earthaccess.download(links_list, path)
    return links_list

Extraction des fichiers ZIP et chargement des shapefiles

In [ ]:
# Fonction pour extraire les fichiers ZIP
def extract_files(links_list, path):
    filenames = [link.split("/")[-1] for link in links_list]
    for filename in filenames:
        with zipfile.ZipFile(f"{path}/{filename}", 'r') as zip_ref:
            zip_ref.extractall(path)
    return filenames

# Fonction pour charger les shapefiles
def load_shapefiles(filenames, path):
    filename_shps = [filename.replace('zip', 'shp') for filename in filenames]
    return gpd.GeoDataFrame(pd.concat([gpd.read_file(f"{path}/{shp}") for shp in filename_shps], ignore_index=True))

Filtrage des noeuds et calcul des distances

In [ ]:
# Fonction pour filtrer les données par noeuds et temps
def filter_data_by_nodes(SWOT_HR_df, up_node, dn_node, date=None):
    filtered = SWOT_HR_df[(SWOT_HR_df['node_id'] >= dn_node) &
                          (SWOT_HR_df['node_id'] < up_node) &
                          (SWOT_HR_df['reach_id'] != '72520100361')] #  pour noeud problématique
    if date:
        filtered = filtered[filtered['time_str'].str.contains(date)]
    return filtered.sort_values(['node_id'])

def calculate_distances(SWOT_HR_profil):
    """
    Calcule la distance cumulée à partir de la colonne 'p_length' de SWOT_HR_profil_1.

    """
    delta = SWOT_HR_profil.p_length
    # Initialiser un tableau de zéros pour stocker les distances cumulées
    dist_1=np.zeros((len(SWOT_HR_profil),1))

    # Calcul des distances cumulées
    for n in range(len(SWOT_HR_profil)-1):
      dist_1[n+1]=dist_1[n]+delta.iloc[n]

    return dist_1

Tracé du profil

In [ ]:
# Fonction pour tracer le profil de hauteur d'eau
def plot_profile(profile, dist, label):
    fig, ax = plt.subplots(figsize=(15, 5))
    ax.plot(dist, profile['wse'], marker='o', linestyle='None', label=label)
    ax.set_xlabel('Distance cumulative (m)')
    ax.set_ylabel('Hauteur d\'eau (wse)')
    ax.legend()
    plt.show()
    #Sauvegardez votre image
    plt.savefig('/content/profil.png')

Variables à modifier selon les besoins

In [ ]:
# Chemin du répertoire où les données seront enregistrées et variables à identifier
path = '/content/profil_node'
pass_number    = ["063"]  # Liste de numéros de passage (063, 270, 576, 369)
continent_code = "NA"  # Code du continent (NA pour Amérique du Nord)
temporal_range = '2024-04-04 00:00:00', '2024-08-01 23:59:59'
dn_node = 72520100020381  # ID du noeud en aval
up_node = 72520100020561  # ID du noeud en amont

Exécution complète

In [ ]:
# Téléchargement des données
links_list = download_data(pass_number, continent_code, path, temporal_range)
filenames = extract_files(links_list, path)

# Chargement des shapefiles
SWOT_HR_df = load_shapefiles(filenames, path)
SWOT_HR_df['node_id'] = SWOT_HR_df['node_id'].astype(float)


# Filtrage du profil pour la date du 19 avril 2024
profile_1 = filter_data_by_nodes(SWOT_HR_df, up_node, dn_node, date='2024-04-19')

# Calcul des distances cumulées
dist_1 = calculate_distances(profile_1)

# Tracé du profil
plot_profile(profile_1, dist_1, '19 avril 2024')